In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd



In [5]:
%cd /content/drive/My Drive/Colab Notebooks/IDS561


/content/drive/My Drive/Colab Notebooks/IDS561


In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [7]:
#!wget -v https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

In [8]:
!tar -xvf spark-3.1.2-bin-hadoop3.2.tgz

spark-3.1.2-bin-hadoop3.2/
spark-3.1.2-bin-hadoop3.2/R/
spark-3.1.2-bin-hadoop3.2/R/lib/
spark-3.1.2-bin-hadoop3.2/R/lib/sparkr.zip
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/worker.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/worker/daemon.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/tests/testthat/test_basic.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/shell.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/profile/general.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.Rmd
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/sparkr-vignettes.R
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/doc/index.html
spark-3.1.2-bin-hadoop3.2/R/lib/SparkR/R/
spark-3.1.2-

In [9]:
!pip install -q findspark

Set environment Variables

In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/drive/My Drive/Colab Notebooks/IDS561/spark-3.1.2-bin-hadoop3.2"

Create a local spark session

In [11]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [12]:
from pyspark import SparkContext
sc =SparkContext.getOrCreate()

In [13]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType


df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/IDS561/Amazon_Responded_Oct05.csv',
                    header=True, sep= ',',inferSchema=True, quote="\"", escape="\"", multiLine=True)
# df = spark.read.format("csv") \
#         .option("header", True) \
#         .option('multiline', True) \
#         .load("Amazon_Responded_Oct05.csv")



**TASK1 - Step1 ,2, 3**

In [52]:
df_amazon = df.select("id_str","tweet_created_at","user_verified","favorite_count","retweet_count","text_")
type(df_amazon)

pyspark.sql.dataframe.DataFrame

In [53]:
##TASK1 -- STEP1

rdd_data=df_amazon.rdd.map(list)
amazon_rdd1 = rdd_data.filter(lambda x: (x[2] == True))
amazon_rdd = amazon_rdd1.sortByKey(ascending=False)
print(f'Record Count is: {amazon_rdd.count()}')

##TASK1 -- STEP2 
def date_return(str1):
  str2=str1.split(" ")
  return str2[1] + " " + str2[2]

## making one rdd for only verified users with all the columns
data_amazon_rdd_master = amazon_rdd.map(lambda x: (x[0],date_return(x[1]), x[3], x[4], x[5] ))
data_amazon_rdd_master.toDF().show()


# data_amazon_rdd_master.toDF().toPandas().to_csv(r'/content/drive/My Drive/Colab Notebooks/IDS561/result_intermediate.csv', index = False)
data_amazon_rdd = data_amazon_rdd_master.map(lambda x: (x[1],1))

date_occurences = data_amazon_rdd.reduceByKey(lambda a,b: a+b)
#date_occurences.toDF().show()
date_max_occur = date_occurences.max(lambda x: x[1])

print(date_max_occur[0])
print(date_max_occur[1])
	

Record Count is: 171899
+--------------------+------+---+---+--------------------+
|                  _1|    _2| _3| _4|                  _5|
+--------------------+------+---+---+--------------------+
|'894034669540442113'|Aug 06|  0|0.0|@Amtul457BMo That...|
|'893923788785635328'|Aug 05|  0|0.0|@Kietkatchunky Th...|
|'893915893507469313'|Aug 05|  0|0.0|@Kietkatchunky I'...|
|'893879449380425728'|Aug 05|  0|0.0|@kaimal_hari Hi, ...|
|'893732158732414977'|Aug 05|  0|0.0|@netajee Please d...|
|'893732017887694848'|Aug 05|  0|0.0|@netajee I'm afra...|
|'893674308022697984'|Aug 05|  0|0.0|@netajee We would...|
|'893580949094633472'|Aug 04|  0|0.0|@joeandnic Please...|
|'893533358340411392'|Aug 04|  0|0.0|@netajee You can ...|
|'893435728339898368'|Aug 04|  0|0.0|@netajee Also, pl...|
|'893435600791117824'|Aug 04|  0|0.0|@netajee Print yo...|
|'893374293048012800'|Aug 04|  0|0.0|@kaimal_hari Coul...|
|'893238665375350785'|Aug 03|  0|0.0|@DirtyOld_Case Oh...|
|'893233906123780097'|Aug 03|  0

## Task1 -- step3


In [55]:
import pandas as pd

# For date with maximum occurances find the sum of favorite count & retweet count

def replace_na_with_0(val):
  return val if val is not None else 0

# Keeping only integer values
max_occur_rdd = data_amazon_rdd_master.filter(lambda x:x[4] is not None).filter(lambda x:x[0] is not None).filter(lambda x: x[1] == date_max_occur[0]).map(lambda x: (x[0], x[1], replace_na_with_0(x[2]), replace_na_with_0(x[3])))
# max_occur_rdd.toDF().show()
filter_new_data1 = max_occur_rdd.map(lambda x: (x[0], int(x[2]), int(x[3])))
# filter_new_data1.toDF().show()

## Add Fav & RTs
top_occur = filter_new_data1.map(lambda x: (x[0], x[1] + x[2])).reduceByKey(lambda a,b: a+b)

## sort in descending order
top_tweets = top_occur.sortBy(lambda x: x[1], ascending=False)

## keep only top 100
top_100 = top_tweets.take(100)

# Convert list to RDD
rdd_top100 = spark.sparkContext.parallelize(top_100)

# # ## Keeping only the tweet id and text from the initial rdd
rdd_data_2 = rdd_data.map(lambda x: (x[0], x[5]))

# # ## Top 100 tweets with text - RDD 
final_top100 = rdd_top100.join(rdd_data_2).map(lambda x: (x[0], x[1][1]))
final_top100.toDF().count()




100

In [62]:
import re

# Remove special characters from tweets
# Treating existenxe of a special character differently
def rem_special_chrs(s):
  if s:
    rs = re.sub('[^a-zA-Z\s]', '', s)
    return re.sub('[.,]', '', rs)
  else:
    return None

def printrdd(rdd1):
  rdd1.toDF().show()

## Only Texts from the rdd

words_rdd = final_top100.map(lambda x: (x[0], rem_special_chrs(x[1]))).filter(lambda x:x[1] is not None).map(lambda x:(x[0], x[1].lower()))
printrdd(words_rdd)

## removing special characters, converting to lower, etc - Steps for DATA CLEANING
fin_rdd = words_rdd.flatMap(lambda x:  x[1].split()).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False)
fin_rdd.collect()
printrdd(fin_rdd)

#final_rdd = fin_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b).sortByKey()
# final_rdd.sortBy(lambda x: -x[1]).collect()

# ## Convert into Dataframe
FinalDF = fin_rdd.toDF()

# # # Write to CSV
Final_csv1 = FinalDF.withColumnRenamed("_1", "Word").withColumnRenamed("_2", "Frequency")
Final_csv = Final_csv1.sort(Final_csv1.Frequency.desc())
Final_csv.toPandas().to_csv(r'/content/drive/My Drive/Colab Notebooks/IDS561/IDS561HW2WordCountResult_Anisha.csv', index = False)

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|'816314295680110593'|thedexterouz hi w...|
|'816095108013654017'|kstefl sounds lik...|
|'816433714234355713'|lisadeee you can ...|
|'816426147537965056'|gxb im sorry to h...|
|'816416972422451201'|bbyuniversecb if ...|
|'816412707473727488'|profgambs hi im s...|
|'816397476093657090'|bayou glad we wer...|
|'816381754021974016'|eumaeusodyssey ke...|
|'816378601629024262'|katmcor we should...|
|'816371263698509824'|danadug im sorry ...|
|'816365998060294148'|chunkkyyy courier...|
|'816361991484964866'|uoduckscotty if s...|
|'816353313633476608'|gerzinal pawsome ...|
|'816344377522065408'|danjbalkwill im s...|
|'816332801138184193'|stadesse sorry fo...|
|'816327758548701184'|dragonflyeye  im ...|
|'816307885168852993'|tomroe most carri...|
|'816307884791316480'|darshanhsheth hey...|
|'816300335039938560'|darshanhsheth hi ...|
|'816290520213364736'|darshanhsh

**TASK2 - Fill the text Column**

In [63]:
## read the file
df_textfile = spark.read.csv('find_text.csv',
                    header=True, sep= ',',inferSchema=True, quote="\"", escape="\"", multiLine=True)


In [64]:
import pandas as pd

df_textfile.head(10)

## Convert the text file into RDD
text_rdd = df_textfile.rdd.map(list)

text_rdd_id = text_rdd.map(lambda x: (x[0],"NA"))
#text_rdd_id.collect()
#rdd_data.collect()
id_and_text_rdd = rdd_data.map(lambda x: (x[0], x[5]))
id_and_text_rdd.collect()
# type(text_rdd)
# type(data_amazon_rdd_master)

## JOIN The two RDDs
final_rdd2 = text_rdd_id.join(id_and_text_rdd).map(lambda x:(x[0],x[1][1]))

## Convert into Dataframe
FinalDF2 = spark.createDataFrame(final_rdd2)

# Write to CSV
Final_csv2 = FinalDF2.withColumnRenamed("_1", "Twitter ID").withColumnRenamed("_2", "Tweet Text")
Final_csv2.toPandas().to_csv(r'/content/drive/My Drive/Colab Notebooks/IDS561/IDS561HW2_Text_Find_Result_Anisha.csv', index = False)